# Hasura GraphQL Engine Demo

## Instant GraphQL APIs on Your Data

---

**What we'll cover:**
- What is Hasura?
- GraphQL API Generation
- Database Migrations & Schema Management
- Real-time Subscriptions
- Role-based Access Control
- Latest Features (v2.48+ and v3 Preview)

# What is Hasura?

Hasura is an **open-source GraphQL engine** that:

- Connects to your databases and services
- Automatically generates a **real-time GraphQL API**
- Provides **fine-grained access control**
- Handles **database migrations** and schema management
- Supports **event triggers** and webhooks

### Supported Data Sources
- PostgreSQL (primary)
- SQL Server, MySQL, BigQuery
- Remote GraphQL & REST APIs

# Setup: Installing Dependencies

First, let's install the required Python packages for interacting with our Hasura GraphQL API.

In [1]:
!pip install requests pandas tabulate python-graphql-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.9/185.9 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.4/368.4 kB 85.2 MB/s eta 0:00:00


In [2]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML, Markdown

# Hasura GraphQL Endpoint
HASURA_URL = "http://hasura:8080/v1/graphql"
HASURA_ADMIN_SECRET = "hasura-admin-secret"

def execute_graphql(query, variables=None, headers=None):
    """Execute a GraphQL query against Hasura"""
    default_headers = {
        "Content-Type": "application/json",
        "X-Hasura-Admin-Secret": HASURA_ADMIN_SECRET
    }
    if headers:
        default_headers.update(headers)
    
    payload = {"query": query}
    if variables:
        payload["variables"] = variables
    
    response = requests.post(HASURA_URL, json=payload, headers=default_headers)
    return response.json()

print("GraphQL client configured!")

GraphQL client configured!


# Feature 1: Instant GraphQL API Generation

Hasura **automatically generates** GraphQL types, queries, and mutations from your database schema.

### What you get instantly:
- `query` - Fetch data with filtering, sorting, pagination
- `mutation` - Insert, update, delete operations
- `subscription` - Real-time data streaming

In [5]:
# Example: Query all products with their categories
query = """
query GetProducts {
  products(order_by: {price: desc}, limit: 5) {
    name
    price
    stock_quantity
    category {
      name
    }
  }
}
"""

result = execute_graphql(query)
products = result.get('data', {}).get('products', [])

# Display as a nice table
df = pd.DataFrame([
    {
        'Product': p['name'],
        'Price': f"${p['price']}",
        'Stock': p['stock_quantity'],
        'Category': p['category']['name'] if p.get('category') else 'N/A'
    }
    for p in products
])
display(df)

""


# Feature 2: Powerful Filtering & Aggregations

Hasura provides **built-in operators** for complex queries:

| Operator | Description |
|----------|-------------|
| `_eq`, `_neq` | Equal / Not equal |
| `_gt`, `_gte`, `_lt`, `_lte` | Comparison operators |
| `_in`, `_nin` | In array / Not in array |
| `_like`, `_ilike` | Pattern matching |
| `_is_null` | Null check |
| `_and`, `_or`, `_not` | Boolean operators |

In [6]:
# Complex filtering example: Products in Electronics category with price > $1000
query = """
query FilteredProducts {
  products(
    where: {
      _and: [
        { price: { _gte: 1000 } },
        { category: { name: { _ilike: "%computer%" } } }
      ]
    }
  ) {
    name
    price
    metadata
  }
}
"""

result = execute_graphql(query)
print(json.dumps(result, indent=2))

{
  "errors": [
    {
      "message": "field 'products' not found in type: 'query_root'",
      "extensions": {
        "path": "$.selectionSet.products",
        "code": "validation-failed"
      }
    }
  ]
}


In [ ]:
# Aggregation example: Get order statistics
query = """
query OrderStats {
  orders_aggregate {
    aggregate {
      count
      sum {
        total_amount
      }
      avg {
        total_amount
      }
    }
  }
  
  orders_aggregate(where: {status: {_eq: "delivered"}}) {
    aggregate {
      count
    }
  }
}
"""

result = execute_graphql(query)
print("Order Statistics:")
print(json.dumps(result, indent=2))

# Feature 3: Relationships & Joins

Hasura automatically detects **foreign key relationships** and creates:

- **Object relationships** (many-to-one): e.g., `order.user`
- **Array relationships** (one-to-many): e.g., `user.orders`

You can also create **manual relationships** for more complex scenarios.

In [ ]:
# Nested query with multiple relationships
query = """
query OrdersWithDetails {
  orders {
    id
    status
    created_at
    
    # Object relationship: Order -> User
    user {
      name
      email
    }
    
    # Array relationship: Order -> Order Items
    order_items {
      quantity
      unit_price
      
      # Nested: Order Item -> Product
      product {
        name
      }
    }
  }
}
"""

result = execute_graphql(query)
print(json.dumps(result, indent=2))

# Feature 4: Mutations (Insert, Update, Delete)

Hasura generates mutation operations automatically:

- `insert_<table>` / `insert_<table>_one`
- `update_<table>` / `update_<table>_by_pk`
- `delete_<table>` / `delete_<table>_by_pk`

In [ ]:
# Insert a new user
mutation = """
mutation CreateUser($email: String!, $name: String!) {
  insert_users_one(
    object: {
      email: $email,
      name: $name,
      role: "customer"
    }
  ) {
    id
    email
    name
    created_at
  }
}
"""

variables = {
    "email": "newuser@example.com",
    "name": "New Demo User"
}

result = execute_graphql(mutation, variables)
print(json.dumps(result, indent=2))

In [ ]:
# Upsert example (insert or update on conflict)
mutation = """
mutation UpsertUser {
  insert_users_one(
    object: {
      email: "newuser@example.com",
      name: "Updated Demo User"
    },
    on_conflict: {
      constraint: users_email_key,
      update_columns: [name]
    }
  ) {
    id
    email
    name
  }
}
"""

result = execute_graphql(mutation)
print("Upsert result:")
print(json.dumps(result, indent=2))

# Feature 5: Role-Based Access Control (RBAC)

Hasura provides **fine-grained permissions** at the row and column level.

### Permission Types:
- **Select** - Who can read what rows/columns
- **Insert** - Who can create with what values
- **Update** - Who can modify which rows/columns
- **Delete** - Who can delete which rows

### Session Variables:
- `X-Hasura-User-Id`
- `X-Hasura-Role`
- Custom variables...

In [ ]:
# Simulating a customer role - can only see their own orders
# In production, this would be set by your auth system

query = """
query MyOrders {
  orders {
    id
    status
    total_amount
    created_at
  }
}
"""

# With admin access - sees all orders
print("=== Admin view (all orders) ===")
admin_result = execute_graphql(query)
print(f"Total orders visible: {len(admin_result.get('data', {}).get('orders', []))}")

# With customer role - would only see their orders (requires permission setup)
# customer_headers = {
#     "X-Hasura-Role": "customer",
#     "X-Hasura-User-Id": "550e8400-e29b-41d4-a716-446655440003"
# }
# customer_result = execute_graphql(query, headers=customer_headers)

# Feature 6: Database Migrations & Schema Management

Hasura CLI provides powerful migration tools:

```bash
# Initialize Hasura project
hasura init my-project --endpoint http://localhost:8080

# Create a new migration
hasura migrate create add_user_profile --from-server

# Apply migrations
hasura migrate apply

# Export metadata
hasura metadata export

# Apply metadata
hasura metadata apply
```

### Benefits:
- Version-controlled schema changes
- Reproducible environments
- CI/CD integration

## Migration File Structure

```
hasura/
├── config.yaml
├── metadata/
│   ├── databases/
│   │   └── default/
│   │       └── tables/
│   │           ├── public_users.yaml
│   │           └── public_orders.yaml
│   ├── actions.yaml
│   └── remote_schemas.yaml
├── migrations/
│   └── default/
│       ├── 1234567890_init/
│       │   ├── up.sql
│       │   └── down.sql
│       └── 1234567891_add_reviews/
│           ├── up.sql
│           └── down.sql
└── seeds/
    └── default/
        └── seed_data.sql
```

# Feature 7: Real-time Subscriptions

Hasura supports **GraphQL subscriptions** out of the box:

```graphql
subscription WatchOrders {
  orders(where: {status: {_eq: "pending"}}) {
    id
    status
    total_amount
    user {
      name
    }
  }
}
```

### Use Cases:
- Live dashboards
- Real-time notifications
- Collaborative applications
- Live feeds and chat

# Feature 8: Event Triggers & Webhooks

Hasura can trigger webhooks on database events:

| Event | Description |
|-------|-------------|
| INSERT | When a new row is created |
| UPDATE | When a row is modified |
| DELETE | When a row is removed |

### Example Use Cases:
- Send welcome email on user signup
- Update inventory when order is placed
- Sync data to external systems
- Trigger notifications

In [ ]:
# Event trigger payload example
event_payload_example = {
    "event": {
        "op": "INSERT",
        "data": {
            "old": None,
            "new": {
                "id": "550e8400-e29b-41d4-a716-446655440099",
                "email": "newuser@example.com",
                "name": "New User",
                "created_at": "2024-01-15T10:30:00Z"
            }
        }
    },
    "table": {
        "schema": "public",
        "name": "users"
    },
    "trigger": {
        "name": "user_created"
    }
}

print("Example Event Trigger Payload:")
print(json.dumps(event_payload_example, indent=2))

# Feature 9: Actions (Custom Business Logic)

**Actions** allow you to extend your GraphQL schema with custom logic:

```graphql
type Mutation {
  processPayment(orderId: uuid!, amount: Float!): PaymentResult
}

type PaymentResult {
  success: Boolean!
  transactionId: String
  message: String
}
```

### Benefits:
- Write business logic in any language
- Integrate with external APIs
- Keep complex logic outside the database
- Serverless function support

# Latest Hasura Features (v2.48+)

## PostgreSQL 17 Support
- Full compatibility with latest PostgreSQL
- Improved `pg_dump` output format

## Enhanced Streaming Subscriptions
- `remove_empty_subscription_responses` now works on streaming subscriptions
- Reduces data transfer for highly-multiplexed queries

## New Configuration Options
- `HASURA_GRAPHQL_PRESERVE_401_ERRORS` - Propagate 401 status codes
- `HASURA_GRAPHQL_SERVER_TIMEOUT` - Configure request timeouts

## Security Improvements
- Removed `value_from_env` from event trigger logs
- Protected sensitive header values from leaking

# Hasura v3 Engine Preview

## Major Architecture Changes

- **Rewritten in Rust** (previously Haskell)
  - Better performance and scalability
  - More accessible codebase for contributors

## Native Data Connectors (NDC)
- Standardized connector specification
- Available: PostgreSQL, ClickHouse, Deno (TypeScript)
- Coming: MongoDB, DynamoDB, MySQL, Oracle

## New Metadata Objects
- **Models** - Queryable data collections
- **Commands** - Functions with business logic
- **Relationships** - Cross-source data linking
- **Global IDs** - Relay specification support

# Hasura Data Delivery Network (DDN)

Built on v3 engine, Hasura DDN is the next-generation platform:

## Key Capabilities
- **Federated Data Access** - Connect multiple data sources
- **Edge Deployment** - Lower latency, global distribution
- **Enhanced Security** - Fine-grained access control
- **PromptQL Integration** - AI-native data access

## PromptQL (2025)
- Natural language queries on your data
- AI-powered data exploration
- Seamless GraphQL integration

# Demo: Working with JSONB Data

In [ ]:
# Hasura supports JSONB queries natively
query = """
query ProductsByBrand {
  products(
    where: {
      metadata: { _contains: { brand: "Apple" } }
    }
  ) {
    name
    price
    metadata
  }
}
"""

result = execute_graphql(query)
print("Apple Products:")
print(json.dumps(result, indent=2))

# Demo: Views as GraphQL Endpoints

In [ ]:
# PostgreSQL views are automatically exposed as GraphQL queries
query = """
query ProductStatistics {
  product_stats(order_by: {avg_rating: desc}, limit: 5) {
    name
    price
    review_count
    avg_rating
    total_sold
  }
}
"""

result = execute_graphql(query)
if 'data' in result and result['data'].get('product_stats'):
    df = pd.DataFrame(result['data']['product_stats'])
    display(df)
else:
    print("Note: Track the product_stats view in Hasura Console to enable this query")
    print(json.dumps(result, indent=2))

# Best Practices Summary

## Development
1. Use **migrations** for all schema changes
2. Export and version-control **metadata**
3. Use **seeds** for consistent test data
4. Set up **CI/CD pipelines** with Hasura CLI

## Security
1. Always define **row-level permissions**
2. Use **column presets** for audit fields
3. Enable **rate limiting** in production
4. Use **JWT/webhook authentication**

## Performance
1. Create appropriate **database indexes**
2. Use **views** for complex aggregations
3. Monitor with **Hasura metrics**
4. Consider **read replicas** for scaling

# Resources

## Official Documentation
- [Hasura Docs](https://hasura.io/docs/)
- [GraphQL Engine GitHub](https://github.com/hasura/graphql-engine)
- [Hasura Blog](https://hasura.io/blog/)

## Community
- [Discord Community](https://discord.gg/hasura)
- [GitHub Discussions](https://github.com/hasura/graphql-engine/discussions)

## This Demo
- Hasura Console: http://localhost:8080
- Admin Secret: `hasura-admin-secret`

# Thank You!

## Questions?

---

### Try it yourself:

```bash
# Start the demo
docker-compose up -d

# Access Hasura Console
open http://localhost:8080

# Admin Secret
hasura-admin-secret
```